In [ ]:
class AsyncRestApiGet:

    def __init__(self, number_concurrent_requests, BASE_URL, HEADERS, list_extension_id):
        self.number_concurrent_requests = number_concurrent_requests
        self.BASE_URL = BASE_URL
        self.HEADERS = HEADERS
        self.list_extension_id = list_extension_id

    # Asynchronous function to fetch data from Fleetio API
    async def fetch_data(self, session, url):
        SEMAPHORE = asyncio.Semaphore(self.number_concurrent_requests)
        async with SEMAPHORE:  # Limit concurrent requests
            try:
                async with session.get(url, headers=self.HEADERS or {}) as response:
                    if response.status == 200:
                        return await response.json()
                    else:
                        error_message = await response.text()
                        print(f"Error {response.status} for {url}: {error_message}")
                        return None
            except Exception as e:
                print(f" Request failed for {url}: {e}")
                return None

    # Asynchronous main function to fetch data for all IDs
    async def main(self):
        
        urls = [self.BASE_URL+f"{form_id}" for form_id in self.list_extension_id]

        async with aiohttp.ClientSession() as session:
            tasks = [self.fetch_data(session, url) for url in urls]
            results = await asyncio.gather(*tasks)
        if results is not None:
            print(f"Fetched Data for {self.BASE_URL}")
        return results
    
    # run main fucntion
    def get_data(self):
        loop = asyncio.get_event_loop()
        data= loop.run_until_complete(self.main())
        return data

In [ ]:
async_api_data= AsyncRestApiGet(
   number_concurrent_requests=5, 
   BASE_URL= f"https://orangegamesapi/api/v1/submitted_forms/", 
   HEADERS=HEADERS, list_extension_id= ["69033893", "69033882", "69033863"]
)

dump= async_api_data.get_data()
dump